In [1]:
import numpy as np
import pandas as pd
import re

class Preprocessing:
    def __init__(self, file_path):
        """
        Initialize the Preprocessing class with a file path and load it into a DataFrame.

        Parameters:
        file_path: str
            Path to the file to be processed.
        """
        self.file_path = file_path
        self.df = pd.read_csv(file_path)

    @staticmethod
    def return_null(val):
        """
        Replace values containing '__' or '_' in a string with nan.

        Parameters:
        val: The input value to be checked.

        Returns:
        nan if the condition is met, otherwise the original value.
        """
        if isinstance(val, str) and ("__" in val or "_" in val):
            return np.nan
        return val

    def process_invalid_numerical_columns(self):
        """
        Process specified numerical columns by replacing invalid values with nan and converting to float.

        Returns:
        pandas.DataFrame
            A new DataFrame with processed numerical columns based on the loaded CSV file.
        """
        new_df = self.df.copy()
        invalid_numerical_columns = [
            'Age',
            'Annual_Income',
            'Monthly_Inhand_Salary',
            'Num_of_Loan',
            'Num_of_Delayed_Payment',
            'Changed_Credit_Limit',
            'Outstanding_Debt',
            'Amount_invested_monthly',
            'Monthly_Balance'
        ]

        for col in invalid_numerical_columns:
            if col in new_df.columns:
                new_df[col] = new_df[col].apply(self.return_null)
                new_df[col] = new_df[col].astype('float')

        # Additional processing
        new_df["Age"] = new_df.Age.apply(lambda x: np.nan if x < 0 or x > 90 else x)
        new_df["Num_Bank_Accounts"] = new_df.Num_Bank_Accounts.apply(lambda x: np.nan if x < 0 or x > 15 else x)
        new_df["Num_Credit_Card"] = new_df.Num_Credit_Card.apply(lambda x: np.nan if x < 0 or x > 15 else x)
        new_df["Interest_Rate"] = new_df.Interest_Rate.apply(lambda x: np.nan if x > 50 else x)
        new_df["Num_of_Loan"] = new_df.Num_of_Loan.apply(lambda x: np.nan if x < 0 or x > 10 else x)
        new_df["Delay_from_due_date"] = new_df.Delay_from_due_date.apply(lambda x: 0.0 if x < 0 else x)
        new_df["Num_of_Delayed_Payment"] = new_df.Num_of_Delayed_Payment.apply(lambda x: np.nan if x < 0 or x > 30 else x)
        new_df["Changed_Credit_Limit"] = new_df.Changed_Credit_Limit.apply(lambda x: 0.0 if x < 0 else x)
        new_df["Num_Credit_Inquiries"] = new_df.Num_Credit_Inquiries.apply(lambda x: np.nan if x > 25 else x)
        new_df["Total_EMI_per_month"] = new_df.Total_EMI_per_month.apply(lambda x: np.nan if x > 1400 else x)

        return new_df

    @staticmethod
    def cal_history_age(val):
        """
        Calculate history age in months based on a string value containing years and months.

        Parameters:
        val: str
            The input value to be parsed.

        Returns:
        int or nan
            Total months calculated or nan if parsing fails.
        """
        try:
            year, month = 0, 0
            if re.search('year', val, re.IGNORECASE):
                year = int(re.findall('\d+', val)[0])
            if re.search('month', val, re.IGNORECASE):
                month = int(re.findall('\d+', val)[1])
            return year * 12 + month
        except:
            return np.nan

<>:88: SyntaxWarning: invalid escape sequence '\d'
<>:90: SyntaxWarning: invalid escape sequence '\d'
<>:88: SyntaxWarning: invalid escape sequence '\d'
<>:90: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Dicoding\AppData\Local\Temp\ipykernel_34904\3759201137.py:88: SyntaxWarning: invalid escape sequence '\d'
  year = int(re.findall('\d+', val)[0])
C:\Users\Dicoding\AppData\Local\Temp\ipykernel_34904\3759201137.py:90: SyntaxWarning: invalid escape sequence '\d'
  month = int(re.findall('\d+', val)[1])


In [ ]:
# Example usage:
file_path = "train.csv"
preprocessor = Preprocessing(file_path)

# Process the DataFrame loaded from the file path
new_df = preprocessor.process_invalid_numerical_columns()
new_df['Credit_History_Age'] = new_df['Credit_History_Age'].apply(Preprocessing.cal_history_age)
new_df.drop(["ID", "Name", "SSN", "Annual_Income"], axis=1, inplace=True)

numerical_columns = [
    'Age',
    'Monthly_Inhand_Salary',
    'Num_Bank_Accounts',
    'Num_Credit_Card',
    'Interest_Rate',
    'Num_of_Loan',
    'Delay_from_due_date',
    'Num_of_Delayed_Payment',
    'Changed_Credit_Limit',
    'Num_Credit_Inquiries',
    'Outstanding_Debt',
    'Credit_Utilization_Ratio',
    'Total_EMI_per_month',
    'Amount_invested_monthly',
    'Monthly_Balance',
    'Credit_History_Age'
]

def fill_numerical_missing_value(column, how, df):
    df = df.copy()
    
    missing_customer_id = df[df[column].isna()].Customer_ID
    new = df[df.Customer_ID.isin(missing_customer_id)].groupby(by="Customer_ID", as_index=False).agg({
        column: how
    })
    
    for index, row in df[df[column].isna()].iterrows():
        df[column].iloc[index] = new[new.Customer_ID == str(row["Customer_ID"])][column]
 
    return df[column]
 
for col in numerical_columns:
    new_df[col] = fill_numerical_missing_value(
        column=col,
        how="median",
        df=new_df
    )

new_df['Monthly_Balance'] = new_df.groupby('Customer_ID')['Monthly_Balance'].transform(lambda v: v.ffill())
new_df['Monthly_Balance'] = new_df.groupby('Customer_ID')['Monthly_Balance'].transform(lambda v: v.bfill())
new_df['Monthly_Balance'] = new_df['Monthly_Balance'].astype('float')
 
new_df['Credit_History_Age'] = new_df.groupby('Customer_ID')['Credit_History_Age'].transform(lambda v: v.ffill())
new_df['Credit_History_Age'] = new_df.groupby('Customer_ID')['Credit_History_Age'].transform(lambda v: v.bfill())
new_df['Credit_History_Age'] = new_df['Credit_History_Age'].astype('float')

categorical_columns = [
    'Month',
    'Occupation',
    'Type_of_Loan',
    'Credit_Mix',
    'Payment_of_Min_Amount',
    'Payment_Behaviour',
    'Credit_Score'
]

new_df['Occupation'] = new_df['Occupation'].apply(lambda x: np.nan if x == "_______" else x)
new_df['Credit_Mix'] = new_df['Credit_Mix'].str.replace('_','Unknown')
new_df['Payment_Behaviour'] = new_df['Payment_Behaviour'].str.replace('!@9#%8','Unknown')
 
new_df['Type_of_Loan'] = new_df[~(new_df.Type_of_Loan.isna())]['Type_of_Loan'].apply(lambda x: str(x).split(',')[0])

new_df['Occupation'] = new_df.groupby('Customer_ID')['Occupation'].transform(lambda v: v.ffill())
new_df['Occupation'] = new_df.groupby('Customer_ID')['Occupation'].transform(lambda v: v.bfill())
 
new_df["Type_of_Loan"].fillna(value="Not Specified", inplace=True)

new_df

In [ ]:
new_df.to_csv('train_cleaned.csv', index=False)